# Eventdata und AMP-Berechnung für FUR/WET

In [1]:
import os
import gc
from functions.hilfsfunktionen import transform_to_rtz
import obspy as obs
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from obspy.geodetics import gps2dist_azimuth, locations2degrees
from pprint import pprint
from functions.add_distances_and_backazimuth import __add_distances_and_backazimuth
from functions.querrySeismoData import __querrySeismoData
from obspy import read
import glob
import sys
from scipy import fftpack
from andbro__fft import __fft
from obspy.taup import TauPyModel
from functions.get_octave_bands import __get_octave_bands

### Methods

In [2]:
from obspy.taup import TauPyModel
def getStarttime(origin_time,distance_deg, ev_depth):
    model = TauPyModel(model="prem")
    arrivals = model.get_travel_times(source_depth_in_km=ev_depth,
                                  distance_in_degree=distance_deg, phase_list=["P"])
    p_arrival_time = arrivals[0].time  # Zeit in Sekunden relativ zum origin_time
    starttime = origin_time + p_arrival_time - 20
    return starttime

In [3]:
#nur für translation --> nicht für rotation
def __makeplotspectraTsmooth(st, config, freq_bands):
    st_in = st.select(component="T")
    if len(st_in) == 0:
        print(" -> keine T-Komponente gefunden.")
        return
        
    tr1, tr2 = None, None
    
    # Stationen gezielt zuordnen
    for tr in st_in:
        if tr.stats.station.upper() == "FUR":
            tr1 = tr
            data1 = tr1.data 
            spec1, ff1, ph1 = __fft(data1, tr.stats.delta, window=None, normalize=None)
        elif tr.stats.station.upper() == "WET":
            tr2 = tr
            data2 = tr2.data
            spec2, ff2, ph2 = __fft(data2, tr.stats.delta, window=None, normalize=None)
        
    spec1_s = spec1# pd.Series(spec1).rolling(window=20, center=True).mean().fillna(method='bfill').fillna(method='ffill')
    spec2_s = spec2#pd.Series(spec2).rolling(window=20, center=True).mean().fillna(method='bfill').fillna(method='ffill')

    max_amplitudes_spec1 = []
    for fmin, fmax in freq_bands:
        mask = (ff1 >= fmin) & (ff1 <= fmax)
        if np.any(mask):
            perc95 = np.percentile(spec1_s[mask], 95)
            max_amplitudes_spec1.append(perc95)
        else:
            max_amplitudes_spec1.append(np.nan)
    max_amplitudes_spec2 = []
    for fmin, fmax in freq_bands:
        mask = (ff2 >= fmin) & (ff2 <= fmax)
        if np.any(mask):
            perc95 = np.percentile(spec2_s[mask], 95)
            max_amplitudes_spec2.append(perc95)
        else:
            max_amplitudes_spec2.append(np.nan)

    result_dict = {
            "origin_time": str(origin_time),
            }
    for i, center in enumerate(band_centers):
        result_dict[f"amp1_{center:.4f}Hz"] = max_amplitudes_spec1[i]
        result_dict[f"amp2_{center:.4f}Hz"] = max_amplitudes_spec2[i]
    del st_in
    return result_dict

### Configurations

In [4]:
config = {}

# ROMY coordinates
config['ROMY_lon'] = 11.275501
config['ROMY_lat'] = 48.162941

# duration of event in seconds
config['duration'] = 7200

# frequency range for bandpass filter
config['fmin'] = 0.01 # in Hz   urspr. 0.01
config['fmax'] = 1 # in Hz    urspr. 0.1

# path for figures to store
config['outpath_figs'] = "C:/Bachelorarbeit/figures/acc/FURWET/"

# path for output data
config['outpath_data'] = "C:/Bachelorarbeit/data/waveformsFURWET/"

# specify seed codes of stations that should be used for the analysis
config['seeds'] = ["GR.FUR..BHZ", "GR.FUR..BHN", "GR.FUR..BHE", # seismometer ROMY
                   "GR.WET..BHZ", "GR.WET..BHN", "GR.WET..BHE", # seismometer G
                  ]
# path to catalogs
config['path_to_catalog'] = "C:/Bachelorarbeit/data/catalogs/"

# catalog file
config['catalog'] = "ROMY_global_catalog_20200101_20250531.pkl"

## Load Events

In [5]:
events = pd.read_pickle(config['path_to_catalog']+config['catalog'])

events['origin'] = events.timestamp

# make sure only events with magnitude > 6 are considered
events = events[events.magnitude > 6]
print("Event number: ", events.shape[0])

# avoid events that are too close to each other in time
events['elapsed_time'] = events.timestamp.diff()
events = events[events.elapsed_time > pd.Timedelta(minutes=60)]
print("Event number: ", events.shape[0])

output_path = config['outpath_figs'] + 'Katalog.csv'

# Speichert den DataFrame als CSV-Datei
events.to_csv(output_path, index=False)

Event number:  466
Event number:  385


# RUN LOOP

In [6]:
errors = []
results = []
results_T = []
model = TauPyModel(model="prem")
skip_indices = {3,4,1,2,5,6,9,10,14,20,24,28,33,36,37,40,41,44,45,46,48,49,47,51,56,57,59,60,61,62,72,70,72,71,78,82,86,96,98,99,100,104,105,108,109,110,
                113,115,123,124,125,126,127,128,129,130,131,132,133,135,136,137,144,146,156,157,167,170,171,173,174,175,178,180,181,182,183,184,185,
                189,190,193,196,199,201,202,204,207,208,209,210,211,212,213,215,220,222,223,228,229,230,232,233,237,238,239,240,241,242,245,246,247,248,
                249,250,253,254,255,256,258,259,260,263,264,265,266,268,272,273,274,275,276,278,279,280,281,282,283,284,285,287,288,293,294,297,298,299,
                303,307,311,313,314,318,324,327,328,332,345,346,349,353,355,356,357,358,359,360,364,366,370,372,379,380,384,15,22,27,29,32,52,97,120,
               139,151,161,163,205,227,243,295,306,316,317,319,331,348,369,377,374,383,269,235,216,187,168,159,79,17,0}
                
i = 0
fmin = config['fmin']
fmax = config['fmax']   
fraction_of_octave = 3  

f_lower, f_upper, f_center = __get_octave_bands(fmin, fmax, faction_of_octave=fraction_of_octave, plot=False)
freq_bands = [(round(fl, 5), round(fu, 5)) for fl, fu in zip(f_lower, f_upper)][2:] 
band_centers = [round(fc, 5) for fc in f_center][2:]
print(freq_bands)
# loop over all events
for jj in range(events.shape[0]):
    print(jj)
    if i != jj:
        print(f"i ungleich jj: i={i}, jj={jj}")
        break
    num = str(jj).rjust(3, "0")
    if i in skip_indices:
        i+=1
        continue
    try:
        event_name = str(events.origin.iloc[jj]).replace("-", "").replace(":", "").replace(" ", "_").split(".")[0]
    except:
        print(f" -> {num}: error for {events.origin.iloc[jj]}")
        i += 1
        continue
    # event metadata
    ev_lat = events.latitude.iloc[jj]
    ev_lon = events.longitude.iloc[jj]
    sta_lat = config['ROMY_lat']
    sta_lon = config['ROMY_lon']
    distance_m, az, backazimuth = gps2dist_azimuth(ev_lat, ev_lon, sta_lat, sta_lon)
    distance_km = distance_m / 1000
    ev_depth = events.depth.iloc[jj] / 1000
    origin_time = obs.UTCDateTime(events.origin.iloc[jj])
    distance_deg = locations2degrees(ev_lat, ev_lon, sta_lat, sta_lon)
    arrivals = model.get_travel_times(source_depth_in_km=ev_depth,
                                  distance_in_degree=distance_deg)
    if len(arrivals) == 0:
        print(f"Keine P-Ankunft für Event {jj} (Tiefe {ev_depth} km, Distanz {distance_deg}°)")
        starttime = origin_time
    else:
        p_arrival_time = arrivals[0].time 
        starttime = origin_time + p_arrival_time - 20
    config['tbeg'] = starttime
    if distance_km < 2000:
        config['duration'] = 1000
    elif distance_km < 5000 and distance_km > 2000:
        config['duration'] = 1500
    elif distance_km < 8000 and distance_km > 5000:
        config['duration'] = 2500
    elif distance_km < 10000 and distance_km > 8000:
        config['duration'] = 3000
    elif distance_km < 14000 and distance_km > 10000:
        config['duration'] = 4000
    elif distance_km < 15000 and distance_km > 14000:
        config['duration'] = 4500
    else:
        config['duration'] = 7200
    if i == 85:
        config['duration'] = 4000
    if i == 111:
        config['duration'] = 2500
    if i == 217:
        config['duration'] = 3000
    if i == 226:
        config['duration'] = 1500
    if i == 244 or i == 300:
        config['tbeg'] = starttime-100
    i += 1
    config['tend'] = config['tbeg'] + config['duration']

    #check if data is aleready there
    if os.path.isfile(config['outpath_data']+f"{num}_{event_name}.mseed"):
        st0 = read(config['outpath_data']+f"{num}_{event_name}.mseed")
        data_vorhanden = True
        print(data_vorhanden)
    else:
        data_vorhanden = False
        st0 = obs.Stream()
        print(data_vorhanden)
    
        for seed in config['seeds']:
            try:
                if "FUR" in seed or "WET" in seed:
                    stx, invx = __querrySeismoData(
                        seed_id=seed,
                        starttime=config['tbeg'],
                        endtime=config['tend'],
                        repository='online',
                        path=None,
                        restitute=True,
                        detail=None,
                        fill_value=None,
                        )
                else:
                    print(f" -> {seed} not found")
                    continue
            except Exception as e:
                print(e)
                print(f" -> failed to request {seed} for event: {origin_time}")
                errors.append(f" -> failed to request {seed} for event: {origin_time}")
                continue
    if len(st0) < 6:
        continue
    # sort stream
    st0 = st0.sort()
    st0 = st0.select(station="FUR") + st0.select(station="WET")
    # fill masked data
    for tr in st0:
        if isinstance(tr.data, np.ma.MaskedArray):
            print(f" -> {tr.stats.channel} has masked data. Filled with zeros.")
            tr.data = tr.data.filled(fill_value=0)

    # preprocess
    print(" -> processing data stream ...")
    st1 = st0.copy()
    st1 = st1.detrend("linear")
    st1 = st1.taper(0.002)
    st1 = st1.filter("bandpass", freqmin=config['fmin'], freqmax=config['fmax'], corners=4, zerophase=True)
    print("gefiltert")
    # RTZ transformation pro Station
    st1_rtz = obs.Stream()
    for net_sta in set(f"{tr.stats.network}.{tr.stats.station}" for tr in st1):
        substream = st1.select(network=net_sta.split(".")[0], station=net_sta.split(".")[1])
        components = [tr.stats.channel[-1] for tr in substream]
        if all(c in components for c in ['N', 'E', 'Z']):
            try:
                substream_rtz = transform_to_rtz(substream, azimuth=az)
                st1_rtz += substream_rtz
                print(f" -> transformed {net_sta} to RTZ")
            except Exception as e:
                print(f" -> RTZ failed for {net_sta}: {e}")
                errors.append(f"RTZ failed for {net_sta} in event {num}")
                continue
                st1_rtz += substream
              
        else:
            print(f" -> skipping {net_sta}, missing components: {components}")
            st1_rtz += substream

    # trim streams
    st1_rtz = st1_rtz.trim(config['tbeg'], config['tend'])
    st0 = st0.trim(config['tbeg'], config['tend'])
    if len(st1_rtz) <5:
        continue
    for tr in st1_rtz:
        tr.differentiate()
    # store waveforms
    if not data_vorhanden:
        waveform_filename = f"{num}_{event_name}.mseed"
        if not os.path.isdir(config['outpath_data']):
            print("created: ", config['outpath_data'])
            os.makedirs(config['outpath_data'])

        try:
            st0.write(config['outpath_data'] + waveform_filename)
            print(f" -> stored at: {config['outpath_data'] + waveform_filename}")
        except Exception as e:
            print(f" -> error storing waveform: {e}")
            errors.append(f" -> error storing waveform: {e}")

    results_t = __makeplotspectraTsmooth(st1_rtz, config, freq_bands)
    dt = pd.DataFrame([results_t])
    t_result_path = config['outpath_figs'] + f"T_results/{num}_{event_name}.csv"
    dt.to_csv(t_result_path, index=False)
    del dt, results_t
    gc.collect()

    st0,st1,st1_rtz = None,None,None
    gc.collect()
# show errors
pprint(errors)

[(0.01413, 0.01778), (0.01778, 0.02239), (0.02239, 0.02818), (0.02818, 0.03548), (0.03548, 0.04467), (0.04467, 0.05623), (0.05623, 0.07079), (0.07079, 0.08913), (0.08913, 0.1122), (0.1122, 0.14125), (0.14125, 0.17783), (0.17783, 0.22387), (0.22387, 0.28184), (0.28184, 0.35481), (0.35481, 0.44668), (0.44668, 0.56234), (0.56234, 0.70795), (0.70795, 0.89125), (0.89125, 1.12202)]
0
1
2
3
4
5
6
7
True
 -> processing data stream ...
gefiltert
 -> transformed GR.FUR to RTZ
 -> transformed GR.WET to RTZ
8
True
 -> processing data stream ...
gefiltert
 -> transformed GR.FUR to RTZ
 -> transformed GR.WET to RTZ
9
10
11
True
 -> processing data stream ...
gefiltert
 -> transformed GR.FUR to RTZ
 -> transformed GR.WET to RTZ
12
True
 -> processing data stream ...
gefiltert
 -> transformed GR.FUR to RTZ
 -> transformed GR.WET to RTZ
13
True
 -> processing data stream ...
gefiltert
 -> transformed GR.FUR to RTZ
 -> transformed GR.WET to RTZ
14
15
16
True
 -> processing data stream ...
gefiltert
 -> 

In [7]:
# T-Komponente
t_files = glob.glob(config['outpath_figs'] + "T_results/*.csv")
df_all_T = pd.concat([pd.read_csv(f) for f in t_files], ignore_index=True)
df_all_T.to_csv(config['outpath_figs'] + "amp_ergebnisse_T.csv", index=False)